# Batching

## Rate Limits

* Prevents malicious attacks
* Balance distribution of the user request


## How rate limits occur

* Too many requests
* Too much text in the request

## Avoiding Rate Limits

* Retry
    * Short wait between requests
* Batching
    * Processing multiple messages in one request
* Reducing tokens
    * Quantifying and cutting down the number of tokens

## Retrying

It can be configure using the tenacity library and the retry decorator

The wait options are in seconds

In [18]:
from openai import OpenAI
from dotenv import load_dotenv
from tenacity import (retry, stop_after_attempt, wait_random_exponential)
import json

load_dotenv()
client = OpenAI()


@retry(wait= wait_random_exponential(min=1, max=60),stop=stop_after_attempt(1))
def get_response(model, messages):
    response_value = client.chat.completions.create(
        model= model,
        messages=messages
    )
    return response_value.choices[0].message.content


## Batching

* Get answer in different request/responses
* This approach is useful when the rate limit is due to the timing of the request and not the number of tokens, one way to avoid it is to send requests in batches.

Here are passing the 3 countries at a time

In [19]:
countries = ["United States","Ireland","India"]
message = [
    {
        'role': 'system',
        "content":"""You are given a series of countries and are asked to return the country and capital city. Provide each of the questions with an answer in the response as separate content."""
    }
]

[message.append({'role':'user',"content":f"'{country}'"}) for country in countries]

response = get_response("gpt-4o-mini",message)
print(response)

Country: United States  
Capital City: Washington, D.C.

---

Country: Ireland  
Capital City: Dublin

---

Country: India  
Capital City: New Delhi


## Reducing Tokens

> Use tiktoken

This helps to check for the number of tokens used before sending the request.

In [20]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")
prompt = "Tokens can be full words, or group of characters commonly grouped together: tokenization."

num_token=len(encoding.encode(prompt))
print(num_token)

17
